## 1. Install Required Libraries

Cài đặt các thư viện cần thiết cho inference trên Google Colab:

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"  # Enable extra 30% context lengths

# Check if running on Colab
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm transformers torch
else:
    # Colab-specific installation
    !pip install --upgrade -qqq uv
    try:
        import numpy, PIL
        get_numpy = f"numpy=={numpy.__version__}"
        get_pil = f"pillow=={PIL.__version__}"
    except:
        get_numpy = "numpy"
        get_pil = "pillow"

    try:
        import subprocess
        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_t4 = False

    # Select appropriate vLLM version based on GPU
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install transformers==4.56.2

print("✅ Installation completed!")

## 2. Load Pre-trained Model with Unsloth

Load model đã được fine-tune cho pháp lý tiếng Việt với vLLM fast inference:

In [2]:
from unsloth import FastLanguageModel
import torch

# Model configuration
model_name = "mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1"
max_seq_length = 4096  # Increase for longer legal documents
dtype = torch.float16  # Use float16 for better performance

print(f"🔄 Loading model: {model_name}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,  # Set to True if memory is limited
    fast_inference = True,  # Enable vLLM fast inference
    gpu_memory_utilization = 0.85,  # Adjust based on available GPU memory
)

print("✅ Model loaded successfully!")
print(f"📊 Model info: {model_name}")
print(f"🔤 Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"📏 Max sequence length: {max_seq_length}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 12-03 09:29:51 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-03 09:29:52 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
🔄 Loading model: mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1
INFO 12-03 09:29:56 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 12-03 09:29:56 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 12-03 09:30:16 [config.py:1472] Using max model len 4096
WARNING 12-03 09:30:16 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 12-03 09:30:18 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 12-03 09:30:18 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1', speculative_config=None, tokenizer='mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disab

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/943 [00:00<?, ?B/s]

✅ Model loaded successfully!
📊 Model info: mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1
🔤 Tokenizer vocab size: 128000
📏 Max sequence length: 4096


## 3. Setup System Prompt for Legal Q&A

Thiết lập system prompt và chat template tối ưu cho các câu hỏi pháp lý tiếng Việt:

In [3]:

reasoning_start = "<start_working_out>"
reasoning_end = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

vietnamese_legal_system_prompt = f"""Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}, sau đó xuống 1 dòng, cách 1 dòng.
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Nguyên tắc hoạt động:
1. Cung cấp thông tin pháp lý chính xác theo quy định của pháp luật Việt Nam
2. Trả lời bằng tiếng Việt rõ ràng, dễ hiểu
3. Phân tích vấn đề một cách logic và có cấu trúc
4. Đưa ra lời khuyên thực tế và khả thi
5. Khi cần thiết, đề xuất tham khảo thêm chuyên gia pháp lý

Lưu ý: Thông tin này chỉ mang tính chất tham khảo. Để có lời khuyên chính thức, hãy tham khảo luật sư hoặc cơ quan có thẩm quyền."""

def create_legal_prompt(question):
    """Create a formatted chat prompt for legal questions"""
    return [
        {"role": "system", "content": vietnamese_legal_system_prompt},
        {"role": "user", "content": question}
    ]

# Test the prompt creation
sample_question = "Quy trình ly hôn đơn phương trong pháp luật Việt Nam như thế nào?"
sample_messages = create_legal_prompt(sample_question)

print("✅ System prompt configured!")
print("\n📝 Sample prompt structure:")
for i, msg in enumerate(sample_messages):
    print(f"{i+1}. Role: {msg['role']}")
    print(f"   Content preview: {msg['content'][:100]}...")
    print()

✅ System prompt configured!

📝 Sample prompt structure:
1. Role: system
   Content preview: Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và ph...

2. Role: user
   Content preview: Quy trình ly hôn đơn phương trong pháp luật Việt Nam như thế nào?...



## 4. Basic Inference with vLLM

Thực hiện inference cơ bản với vLLM SamplingParams:

In [8]:
from vllm import SamplingParams
import time

def generate_legal_response(question, temperature=0.7, top_p=0.9, max_tokens=2048):
    """Generate response for a legal question using vLLM"""

    # Create messages
    messages = create_legal_prompt(question)

    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Configure sampling parameters
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        stop=["<|eot_id|>"]  # Stop at end of turn for Llama models
    )

    # Generate response
    start_time = time.time()
    outputs = model.fast_generate(
        [prompt],
        sampling_params=sampling_params,
        lora_request=None
    )
    generation_time = time.time() - start_time

    response = outputs[0].outputs[0].text

    return {
        "question": question,
        "response": response,
        "generation_time": generation_time,
        "tokens_generated": len(tokenizer.encode(response))
    }

def format_legal_response(response_text, line_length=120):
    """Format response text với GRPO structure và xuống dòng phù hợp"""
    import textwrap
    import re

    # Check if response has GRPO format
    has_working_out = "<start_working_out>" in response_text and "<end_working_out>" in response_text
    has_solution = "<SOLUTION>" in response_text and "</SOLUTION>" in response_text

    if has_working_out and has_solution:
        # Extract working out section
        working_out_pattern = r"<start_working_out>(.*?)<end_working_out>"
        working_out_match = re.search(working_out_pattern, response_text, re.DOTALL)

        # Extract solution section
        solution_pattern = r"<SOLUTION>(.*?)</SOLUTION>"
        solution_match = re.search(solution_pattern, response_text, re.DOTALL)

        if working_out_match and solution_match:
            working_out_text = working_out_match.group(1).strip()
            solution_text = solution_match.group(1).strip()

            # Format working out section
            formatted_working_out = textwrap.fill(
                working_out_text,
                width=line_length,
                break_long_words=False,
                break_on_hyphens=False
            )

            # Format solution section
            formatted_solution = textwrap.fill(
                solution_text,
                width=line_length,
                break_long_words=False,
                break_on_hyphens=False
            )

            # Combine with proper formatting
            formatted_response = f"""<start_working_out>
{formatted_working_out}
<end_working_out>

<SOLUTION>
{formatted_solution}
</SOLUTION>"""

            return formatted_response

    # Fallback formatting for non-GRPO responses
    sentences = response_text.split('. ')
    formatted_sentences = []

    for i, sentence in enumerate(sentences):
        # Add period back (except for last sentence if it doesn't end with period)
        if i < len(sentences) - 1:
            sentence += '.'
        elif not sentence.endswith('.') and not sentence.endswith('!') and not sentence.endswith('?'):
            if sentence.strip():  # Only add period if sentence is not empty
                sentence += '.'

        # Wrap long sentences
        if len(sentence) > line_length:
            wrapped = textwrap.fill(sentence, width=line_length,
                                  break_long_words=False, break_on_hyphens=False)
            formatted_sentences.append(wrapped)
        else:
            formatted_sentences.append(sentence)

    # Join sentences with appropriate spacing
    formatted_text = '\n'.join(formatted_sentences)

    # Clean up extra spaces and improve legal document formatting
    formatted_text = formatted_text.replace('\n\n\n', '\n\n')  # Remove triple newlines
    formatted_text = formatted_text.replace(':\n', ':\n')      # Keep colon formatting

    # Add extra line breaks after numbered items for better readability
    formatted_text = re.sub(r'(\d+\.\s)', r'\n\1', formatted_text)  # Add line break before numbers
    formatted_text = re.sub(r'([a-z]\)\s)', r'\n\1', formatted_text)  # Add line break before letters

    # Clean up any leading newlines
    formatted_text = formatted_text.strip()

    return formatted_text

# Test với câu hỏi mẫu về GRPO format
test_question = "Người tham gia giao thông không đội mũ bảo hiểm sẽ bị phạt từ bao nhiêu tiền"

print("🔄 Generating GRPO format response...")
result = generate_legal_response(test_question, temperature=0.3)  # Lower temp for more structured response

# Format response for better readability
formatted_response = format_legal_response(result['response'])

print("✅ Response generated!")
print("\n" + "="*80)
print(f"❓ Câu hỏi: {result['question']}")
print("\n" + "="*80)
print(f"💬 Trả lời (GRPO Format):")
print(formatted_response)
print("\n" + "="*80)
print(f"⏱️  Thời gian: {result['generation_time']:.2f}s")
print(f"🔢 Tokens: {result['tokens_generated']}")
print(f"⚡ Tốc độ: {result['tokens_generated']/result['generation_time']:.1f} tokens/s")

🔄 Generating GRPO format response...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✅ Response generated!

❓ Câu hỏi: Người tham gia giao thông không đội mũ bảo hiểm sẽ bị phạt từ bao nhiêu tiền

💬 Trả lời (GRPO Format):
<start_working_out>
Câu hỏi của người dùng liên quan đến mức phạt đối với hành vi không đội mũ bảo hiểm khi tham gia giao thông. Để trả lời
câu hỏi này, tôi cần tìm thông tin trong văn bản pháp luật liên quan đến quy định về việc sử dụng mũ bảo hiểm khi tham
gia giao thông. Trong văn bản đã cung cấp, có điều khoản quy định rõ ràng về trách nhiệm của người tham gia giao thông
trong việc sử dụng mũ bảo hiểm. Cụ thể, Điều 28 nêu rõ rằng người tham gia giao thông có trách nhiệm sử dụng mũ bảo hiểm
khi tham gia giao thông. Nếu không thực hiện, họ sẽ bị xử phạt theo quy định của pháp luật. Tuy nhiên, văn bản không nêu
cụ thể mức phạt cho hành vi không đội mũ bảo hiểm. Do đó, để có thông tin chính xác về mức phạt, người dùng nên tham
khảo thêm các quy định cụ thể trong Luật Giao thông đường bộ hoặc các văn bản hướng dẫn thi hành luật.
<end_working_out>

<SOL